Real-ESRGAN 같은 AI 기반 업샘플링
작고 흐릿한 로고도 선명하게 복원 가능
특히 아이콘처럼 엣지(edge)와 구조가 뚜렷한 이미지에 효과적

In [7]:
%load_ext autoreload
%autoreload 2

import os
import time
import base64
import shutil
import tempfile
import cairosvg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from pprint import pprint
from typing import List, Dict, Any
from modules.minio import list_files_by_prefix
from modules.minio import (
    list_files_by_prefix,
    list_files_in_directory,
    list_files,
    list_buckets,
    create_bucket,
    create_directory,
    upload_file,
    download_file,
    delete_file,
    head_files
)
from modules.utils import convert_svg_to_png
import modules.minio as minio

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# AWS Icon Detector 데이터셋 이미지 업로드
def upload_training_images():
    """AWS Icon Detector 데이터셋의 이미지들을 MinIO에 업로드"""
    import os
    from pathlib import Path
    from tqdm import tqdm
    
    # 데이터셋 경로 설정
    dataset_root = Path('AWS-Icon-Detector--4')
    train_images_dir = dataset_root / 'train' / 'images'
    test_images_dir = dataset_root / 'test' / 'images'
    
    # 업로드할 이미지 파일들 수집
    image_files = []
    
    # train 이미지들
    for img_file in train_images_dir.glob('*.jpg'):
        image_files.append(('train', img_file))
    
    # test 이미지들  
    for img_file in test_images_dir.glob('*.jpg'):
        image_files.append(('test', img_file))
    
    print(f"총 {len(image_files)}개의 이미지 파일을 발견했습니다.")
    
    # 업로드 실행
    uploaded_count = 0
    failed_count = 0
    
    for split, img_path in tqdm(image_files, desc="이미지 업로드 중"):
        try:
            # MinIO에서의 키 경로 설정
            key = f"training_data/{split}/images/{img_path.name}"
            
            # 파일 업로드
            if minio.upload_file(BUCKET_NAME, str(img_path), key, quiet=True):
                uploaded_count += 1
            else:
                failed_count += 1
                print(f"업로드 실패: {img_path.name}")
                
        except Exception as e:
            failed_count += 1
            print(f"업로드 오류 - {img_path.name}: {e}")
    
    print(f"\n업로드 완료: {uploaded_count}개 성공, {failed_count}개 실패")
    return uploaded_count, failed_count

# 이미지 업로드 실행
upload_training_images()


총 210개의 이미지 파일을 발견했습니다.


이미지 업로드 중: 100%|██████████| 210/210 [00:06<00:00, 32.04it/s]


업로드 완료: 210개 성공, 0개 실패


(210, 0)

In [9]:
# 라벨 파일(.npy)도 함께 업로드
def upload_training_labels():
    """AWS Icon Detector 데이터셋의 라벨 파일들을 MinIO에 업로드"""
    from pathlib import Path
    from tqdm import tqdm
    
    # 데이터셋 경로 설정
    dataset_root = Path('AWS-Icon-Detector--4')
    train_images_dir = dataset_root / 'train' / 'images'
    test_images_dir = dataset_root / 'test' / 'images'
    
    # 라벨 파일들 수집
    label_files = []
    
    # train 라벨들
    for label_file in train_images_dir.glob('*.npy'):
        label_files.append(('train', label_file))
    
    # test 라벨들
    for label_file in test_images_dir.glob('*.npy'):
        label_files.append(('test', label_file))
    
    print(f"총 {len(label_files)}개의 라벨 파일을 발견했습니다.")
    
    # 업로드 실행
    uploaded_count = 0
    failed_count = 0
    
    for split, label_path in tqdm(label_files, desc="라벨 파일 업로드 중"):
        try:
            # MinIO에서의 키 경로 설정
            key = f"training_data/{split}/labels/{label_path.name}"
            
            # 파일 업로드
            if minio.upload_file(BUCKET_NAME, str(label_path), key, quiet=True):
                uploaded_count += 1
            else:
                failed_count += 1
                print(f"업로드 실패: {label_path.name}")
                
        except Exception as e:
            failed_count += 1
            print(f"업로드 오류 - {label_path.name}: {e}")
    
    print(f"\n라벨 업로드 완료: {uploaded_count}개 성공, {failed_count}개 실패")
    return uploaded_count, failed_count

# 라벨 파일 업로드 실행
upload_training_labels()


총 210개의 라벨 파일을 발견했습니다.


라벨 파일 업로드 중: 100%|██████████| 210/210 [00:08<00:00, 24.12it/s]


라벨 업로드 완료: 210개 성공, 0개 실패


(210, 0)

In [10]:
# 데이터셋 설정 파일 업로드 및 확인
def upload_dataset_config():
    """데이터셋 설정 파일을 업로드"""
    from pathlib import Path
    
    config_file = Path('AWS-Icon-Detector--4/data.yaml')
    if config_file.exists():
        key = "training_data/data.yaml"
        if minio.upload_file(BUCKET_NAME, str(config_file), key, quiet=True):
            print("✅ data.yaml 파일 업로드 성공")
        else:
            print("❌ data.yaml 파일 업로드 실패")
    else:
        print("❌ data.yaml 파일을 찾을 수 없습니다")

upload_dataset_config()

# 업로드된 파일 확인
training_files = minio.list_files_by_prefix(BUCKET_NAME, prefix='training_data/')
print(f"📦 업로드된 학습 데이터 파일 수: {len(training_files)}개")


✅ data.yaml 파일 업로드 성공
📦 업로드된 학습 데이터 파일 수: 421개


In [11]:
BUCKET_NAME = 'aws-diagram-object-detection'

minio.check_bucket_connection(BUCKET_NAME)
minio.list_buckets()

[{'Name': 'aws-diagram-object-detection',
  'CreationDate': datetime.datetime(2025, 8, 6, 2, 28, 46, 799000, tzinfo=tzutc())}]

In [12]:
raw_files = minio.list_files('aws-diagram-object-detection')
pprint(raw_files)
print(len(raw_files))

[{'ETag': '"a0995c632643de92537dfd31b685f1ed"',
  'Key': 'training_data/data.yaml',
  'LastModified': datetime.datetime(2025, 8, 6, 13, 1, 0, 328000, tzinfo=tzutc()),
  'Size': 2790,
  'StorageClass': 'STANDARD'},
 {'ETag': '"ca9efc618ecf005527719506f721fe71"',
  'Key': 'training_data/test/images/index32_png.rf.63ec4144cde7c1a15384ffe895b06a31.jpg',
  'LastModified': datetime.datetime(2025, 8, 6, 12, 59, 34, 443000, tzinfo=tzutc()),
  'Size': 28799,
  'StorageClass': 'STANDARD'},
 {'ETag': '"22700503025d1b550cb930c433d61896"',
  'Key': 'training_data/test/images/index34_png.rf.dbe6344e321bb25b34b0f236c8f27fbb.jpg',
  'LastModified': datetime.datetime(2025, 8, 6, 12, 59, 34, 363000, tzinfo=tzutc()),
  'Size': 30125,
  'StorageClass': 'STANDARD'},
 {'ETag': '"cf914883700dba0e21053fcca0b48e22"',
  'Key': 'training_data/test/images/index35_jpg.rf.50a741d3e69dc7587dd41a5deb037afa.jpg',
  'LastModified': datetime.datetime(2025, 8, 6, 12, 59, 34, 604000, tzinfo=tzutc()),
  'Size': 25016,
  'S

In [13]:
raw_file_extensions = {}
for raw_file in raw_files:
    ext = raw_file['Key'].split('.')[-1]  # 마지막 점 기준
    if ext in raw_file_extensions:
        raw_file_extensions[ext] += 1
    else:
        raw_file_extensions[ext] = 1
print(raw_file_extensions)

{'yaml': 1, 'jpg': 210, 'npy': 210}


In [15]:
# 전처리 데이터 버켓 확인
from modules.minio import list_files_by_prefix

# 이미 S3에 업로드된 PNG 목록 (파일명 기준)
uploaded_pngs = {
    Path(f['Key']).name
    for f in list_files_by_prefix(BUCKET_NAME, prefix='preprocessed/')
    if f['Key'].endswith('.png')
}

In [16]:
import os
import time
from datetime import datetime
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from modules.minio import download_file, upload_file
from modules.utils import convert_svg_to_png

# 설정
YOLO_IMAGE_SIZE = (416, 416)
RAW_DIR = os.path.join(os.getcwd(), 'data', 'raw')
PROCESSED_DIR = os.path.join(os.getcwd(), 'data', 'processed')
QUIET = True  # quiet 모드 사용

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)

# 이미 처리된 PNG 파일 목록 (stem 기준)
existing_png_stems = set(p.stem.split('_')[0] for p in Path(PROCESSED_DIR).glob("*.png"))

# 전처리 루프
for idx, raw_file in tqdm(enumerate(raw_files), total=len(raw_files), desc="전처리 진행중", ncols=100):
    key = raw_file['Key']
    ext = key.split('.')[-1].lower()
    key_stem = Path(key).stem.replace('/', '_')

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S_%f')
    base_name = f"{key_stem}_{timestamp}"
    temp_input_path = os.path.join(RAW_DIR, f"{base_name}.{ext}")
    output_filename = f"{base_name}.png"
    temp_output_path = os.path.join(PROCESSED_DIR, output_filename)
    upload_key = f"preprocessed/{output_filename}"

    # ✅ 이미 업로드된 파일이면 건너뜀
    if output_filename in uploaded_pngs:
        continue

    download_file(bucket_name=BUCKET_NAME, key=key, local_path=temp_input_path, quiet=QUIET)

    try:
        if ext in ['svg', 'svg+xml']:
            with open(temp_input_path, 'r', encoding='utf-8') as f:
                svg_content = f.read()
            png_data = convert_svg_to_png(svg_content, output_size=YOLO_IMAGE_SIZE)
            with open(temp_output_path, 'wb') as f:
                f.write(png_data)

        elif ext in ['gif', 'webp', 'bmp', 'jpg', 'jpeg', 'png']:
            with Image.open(temp_input_path) as im:
                im = im.convert('RGBA')
                im = im.resize(YOLO_IMAGE_SIZE)
                im.save(temp_output_path, format='PNG')

        else:
            if not QUIET:
                print(f"⚠️ 확장자 '{ext}' 처리 불가: {key}")
            continue

        # 업로드
        upload_file(bucket_name=BUCKET_NAME, file_path=temp_output_path, key=upload_key, quiet=QUIET)

    except Exception as e:
        if not QUIET:
            print(f"❌ 파일 처리 실패: {key} / 에러: {e}")

    finally:
        os.remove(temp_input_path)
        time.sleep(0.05)

전처리 진행중: 100%|██████████████████████████████████████████████| 421/421 [00:55<00:00,  7.54it/s]


In [17]:
raw_files = minio.list_files('aws-diagram-object-detection')
pprint(raw_files)
print(len(raw_files))

[{'ETag': '"becf7d87546f7f3443bea9e7044f45d5"',
  'Key': 'preprocessed/index100_png.rf.0b3b0491fae93b76f5f10f0b6b88048e_20250806_220120_052079.png',
  'LastModified': datetime.datetime(2025, 8, 6, 13, 1, 20, 256000, tzinfo=tzutc()),
  'Size': 139873,
  'StorageClass': 'STANDARD'},
 {'ETag': '"2a6842ec10cff99b8787f70f8c325119"',
  'Key': 'preprocessed/index101_jpg.rf.711b15e7878901c8966ad59f1770b609_20250806_220120_318418.png',
  'LastModified': datetime.datetime(2025, 8, 6, 13, 1, 20, 419000, tzinfo=tzutc()),
  'Size': 125032,
  'StorageClass': 'STANDARD'},
 {'ETag': '"6baf94560b4e1498ca59f0aa9d8e65b8"',
  'Key': 'preprocessed/index102_png.rf.b355ef65fb0fb3a2303fe84d7d53c71b_20250806_220120_481978.png',
  'LastModified': datetime.datetime(2025, 8, 6, 13, 1, 20, 609000, tzinfo=tzutc()),
  'Size': 174883,
  'StorageClass': 'STANDARD'},
 {'ETag': '"07c2cd51b1134fa05e5ce757e188dc52"',
  'Key': 'preprocessed/index103_png.rf.ea29619677019e8ac754027d646a2b00_20250806_220120_671555.png',
  'La

In [18]:
raw_file_extensions = {}
for raw_file in raw_files:
    ext = raw_file['Key'].split('.')[-1]  # 마지막 점 기준
    if ext in raw_file_extensions:
        raw_file_extensions[ext] += 1
    else:
        raw_file_extensions[ext] = 1
print(raw_file_extensions)

{'png': 210, 'yaml': 1, 'jpg': 210, 'npy': 210}


In [19]:
preprocessed_files = minio.list_files_by_prefix('aws-diagram-object-detection', 'preprocessed')
print(preprocessed_files)
print(len(preprocessed_files))

[{'Key': 'preprocessed/index100_png.rf.0b3b0491fae93b76f5f10f0b6b88048e_20250806_220120_052079.png', 'LastModified': datetime.datetime(2025, 8, 6, 13, 1, 20, 256000, tzinfo=tzutc()), 'ETag': '"becf7d87546f7f3443bea9e7044f45d5"', 'Size': 139873, 'StorageClass': 'STANDARD'}, {'Key': 'preprocessed/index101_jpg.rf.711b15e7878901c8966ad59f1770b609_20250806_220120_318418.png', 'LastModified': datetime.datetime(2025, 8, 6, 13, 1, 20, 419000, tzinfo=tzutc()), 'ETag': '"2a6842ec10cff99b8787f70f8c325119"', 'Size': 125032, 'StorageClass': 'STANDARD'}, {'Key': 'preprocessed/index102_png.rf.b355ef65fb0fb3a2303fe84d7d53c71b_20250806_220120_481978.png', 'LastModified': datetime.datetime(2025, 8, 6, 13, 1, 20, 609000, tzinfo=tzutc()), 'ETag': '"6baf94560b4e1498ca59f0aa9d8e65b8"', 'Size': 174883, 'StorageClass': 'STANDARD'}, {'Key': 'preprocessed/index103_png.rf.ea29619677019e8ac754027d646a2b00_20250806_220120_671555.png', 'LastModified': datetime.datetime(2025, 8, 6, 13, 1, 20, 773000, tzinfo=tzutc())

    Label Studio는 S3-compatible storage 설정을 지원.

    MinIO는 S3 API 호환 → Label Studio에서 외부 스토리지로 연결 가능.

설정 방법:

    Label Studio UI → 프로젝트 → Storage 탭

    Cloud Storage → S3 추가

    다음 정보 입력:

        S3 bucket name: your-bucket

        S3 endpoint: http://localhost:9000 (MinIO 주소)

        Access key / Secret key: MinIO 사용자 계정 정보

        Region: us-east-1 (아무거나 가능)

    Sync storage 누르면 Label Studio 프로젝트에 바로 데이터 import 가능

→ 장점: 파일 복사나 링크 불필요, 라벨링 이후 결과도 S3에 바로 export 가능